In [15]:
import sqlite3
from openai import OpenAI
import gradio as gr
import json

In [13]:
system_message = """
You are a good assistant for an Airline company called MessiasAI.
Give short, precise answers, no more than 1 sentence.
Always be accurate. If you don't know the answer, say "Cannot help, sorry".
"""

In [23]:
client = OpenAI(api_key='ollama', base_url='http://localhost:11434/v1')
MODEL = 'llama3.2'

In [ ]:
DB = "prices.db"

with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute('CREATE TABLE IF NOT EXISTS prices (city TEXT PRIMARY KEY, price REAL)')
    conn.commit()

In [7]:
def get_ticket_price(city):
    print(f'[+] DATABASE TOOL CALLED: Getting price for {city}', flush=True)

    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT price FROM prices WHERE city = ?', (city.lower(),))
        result = cursor.fetchone()

        return f"Ticket price to {city} is ${result[0]}" if result else "No price data available for this city"

def set_ticket_price(city, price):
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('INSERT INTO prices (city, price) VALUES (?, ?) ON CONFLICT(city) DO UPDATE SET price = ?', (city.lower(), price, price))
        conn.commit()

In [10]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [11]:
tools = [{"type": "function", "function": price_function}]

In [8]:
ticket_prices = {"london":799, "paris":899, "tokyo":1400, "sydney":2999}
for city, price in ticket_prices.items():
    set_ticket_price(city, price)

In [9]:
get_ticket_price('tokyo')

[+] DATABASE TOOL CALLED: Getting price for tokyo


'Ticket price to tokyo is $1400.0'

In [17]:
def handle_tool_calls(message):
    responses = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == 'get_ticket_price':
            arguments = json.loads(tool_call.function.arguments)
            city = arguments.get('destination_city')
            price_details = get_ticket_price(city)
            responses.append(
                {
                "role":"tool",
                "content":price_details,
                "tool_call_id":tool_call.id
                }
            )
    return responses

In [21]:
def chat(message, history):
    history = [
        {"role":h["role"], "content":h["content"]} for h in history
    ]
    messages = [{"role":"system", "content": system_message}] + history + [{'role':'user', 'content':message}]
    

    response = client.chat.completions.create(
        model = MODEL,
        messages=messages,
        tools=tools
    )

    while response.choices[0].finish_reason=='tool_calls':
        message = response.choices[0].message
        responses = handle_tool_calls(message)
        messages.append(message)
        messages.extend(responses)
        response = client.chat.completions.create(
            model = MODEL,
            messages=messages,
            tools = tools
        )

    return response.choices[0].message.content


In [24]:
gr.ChatInterface(fn=chat, type='messages').launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


[+] DATABASE TOOL CALLED: Getting price for <destination city>
[+] DATABASE TOOL CALLED: Getting price for Rio de Janeiro
[+] DATABASE TOOL CALLED: Getting price for Tokyo
[+] DATABASE TOOL CALLED: Getting price for Paris
[+] DATABASE TOOL CALLED: Getting price for Tokyo or Paris
[+] DATABASE TOOL CALLED: Getting price for Tokyo
[+] DATABASE TOOL CALLED: Getting price for Paris
